# Process Stochastic Record for Events

The purpose of this notebook is to process the synthetic, 1,000 realization Frio record for events to simulate to look at inundation extent.

## Parameters and Imports

In [1]:
%matplotlib inline

In [2]:
from IPython.display import display, HTML
import os
import datetime as dt
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import shapely

In [3]:
# 24-hour event depths
Old_100yr = 233.7 # depth in mm, 9.2 in
New_25yr = 236.0 # depth in mm, 9.3 in
Est_25yr_Discharge = 180.0 # base event discharge in cubic meters per second

In [4]:
IN_DIR1 = r'C:\Users\nicholas.martin\Documents\Flood_Risk_Model\Frio_Synthetic_Weather'
OUT_DIR = r'C:\Users\nicholas.martin\Documents\Flood_Risk_Model\Frio_Synthetic_Weather\Processed_Outputs'

In [5]:
NumReal = 1000
FileTemp = "Frio_R%d_DF.pickle"

In [6]:
CNStartTS = pd.Timestamp( 2031, 1, 1, 0, )
CNEndTS = pd.Timestamp( 2060, 12, 31, 23, 59, )

## Test File

In [7]:
InFiler = os.path.normpath( os.path.join( IN_DIR1, FileTemp % 1 ) )
curDF = pd.read_pickle( InFiler, compression='zip' )

In [8]:
display( HTML( curDF.head().to_html() ) )

,Tmax_C,Tmin_C,Precip_mm,Tave_C,MonDelta_T,ETo_mm,Def_mm
2024-01-01,27.615095,6.446398,0.0,17.030746,20.410206,2.983201,-2.983201
2024-01-02,28.049767,7.268944,0.0,17.659355,20.410206,3.044339,-3.044339
2024-01-03,30.505297,9.438797,0.0,19.972046,20.410206,3.251329,-3.251329
2024-01-04,24.393387,6.251533,0.0,15.322460,20.410206,2.859076,-2.859076
2024-01-05,21.589312,3.329568,0.0,12.459440,20.410206,2.619751,-2.619751


In [9]:
curDF["DateTime"] = curDF.index
curDF["RealNum"] = 1

In [10]:
curDF = curDF[["RealNum", "DateTime", "Precip_mm",]].copy()

In [11]:
extractDF = curDF[curDF["Precip_mm"] >= New_25yr].copy() 

In [12]:
display( HTML( extractDF.to_html() ) )

,RealNum,DateTime,Precip_mm
2044-01-05,1,2044-01-05,246.863785
2050-01-05,1,2050-01-05,242.915573
2063-01-06,1,2063-01-06,254.184647


In [13]:
extractDF["Discharge_cms"] = extractDF["Precip_mm"] * ( Est_25yr_Discharge / New_25yr )

## Process Realizations for Events

In [14]:
DFLister = list()

In [15]:
for rR in range(1,NumReal+1,1):
    InFiler = os.path.normpath( os.path.join( IN_DIR1, FileTemp % rR ) )
    curDF = pd.read_pickle( InFiler, compression='zip' )
    curDF["DateTime"] = curDF.index
    curDF["RealNum"] = rR
    curDF = curDF[["RealNum", "DateTime", "Precip_mm",]].copy()
    extractDF = curDF[curDF["Precip_mm"] >= New_25yr].copy()
    extractDF["Discharge_cms"] = extractDF["Precip_mm"] * ( Est_25yr_Discharge / New_25yr )
    DFLister.append( extractDF )
# end for

In [16]:
AllEventsDF = pd.concat( DFLister, ignore_index=True )

In [17]:
display( HTML( AllEventsDF.head().to_html() ) )

,RealNum,DateTime,Precip_mm,Discharge_cms
0,1,2044-01-05,246.863785,188.285934
1,1,2050-01-05,242.915573,185.274597
2,1,2063-01-06,254.184647,193.869644
3,2,2048-02-26,245.518372,187.259781
4,2,2051-01-10,248.133163,189.254105


In [18]:
display( HTML( AllEventsDF.tail().to_html() ) )

,RealNum,DateTime,Precip_mm,Discharge_cms
3578,999,2061-01-27,245.074814,186.921478
3579,1000,2028-01-05,245.558990,187.290756
3580,1000,2050-01-09,238.121597,181.618179
3581,1000,2056-01-05,266.281342,203.095947
3582,1000,2061-01-02,328.370911,250.452393


In [19]:
display( HTML( AllEventsDF.describe().to_html() ) )

,RealNum,DateTime,Precip_mm,Discharge_cms
count,3583.000000,3583,3583.000000,3583.000000
mean,500.158805,2046-09-10 02:55:13.647780864,278.532074,212.439713
min,1.000000,2024-01-07 00:00:00,236.024506,180.018692
25%,252.000000,2038-01-27 00:00:00,242.891487,185.256218
50%,496.000000,2047-03-06 00:00:00,251.877472,192.109940
75%,751.000000,2055-01-08 00:00:00,286.403793,218.443581
max,1000.000000,2065-07-23 00:00:00,656.915161,501.036987
std,288.938843,NaN,58.823586,44.865448


In [20]:
DescDF = AllEventsDF.describe().copy()

## Output to Spreadsheet

In [21]:
OutFiler = os.path.normpath( os.path.join( OUT_DIR, "All_Events_Summary.xlsx" ) )
writer = pd.ExcelWriter( OutFiler )
workbook  = writer.book
format1 = workbook.add_format({'num_format': '#,##0.000'})
format2 = workbook.add_format({'num_format': '#,##0.0000'})
format3 = workbook.add_format({'num_format': '#,##0'})
cLabel = "Summary"
DescDF.to_excel( writer, sheet_name=cLabel, )
# adjust columns
writer.sheets[cLabel].set_column( 0, 0, 18 )
for column in DescDF:
    column_width = max(DescDF[column].astype(str).map(len).max()+6, len(column)+6)
    col_idx = DescDF.columns.get_loc(column)
    if column in ["RealNum",]:
        writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, format3)
    elif column in ["Precip_mm", "Discharge_cms"]:
        writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, format1)
# end for
cLabel = "Events"
AllEventsDF.to_excel( writer, sheet_name=cLabel, )
# adjust columns
writer.sheets[cLabel].set_column( 0, 0, 18 )
for column in AllEventsDF:
    column_width = max(AllEventsDF[column].astype(str).map(len).max()+6, len(column)+6)
    col_idx = AllEventsDF.columns.get_loc(column)
    if column in ["RealNum",]:
        writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, format3)
    elif column in ["Precip_mm", "Discharge_cms"]:
        writer.sheets[cLabel].set_column(col_idx+1, col_idx+1, column_width, format1)
# end for
writer.close()